# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import datetime
from citipy import citipy
import json

# Import API key
from config import Google_Maps_Places



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
vacation = pd.read_csv("weather_vacation_output.csv")
vacation.dropna()
vacation.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Max Temp,Lat,Lng,Wind Speed
0,0,lesnoye,100,RU,1579466463,86,36.88,58.28,35.52,12.41
1,1,tiksi,100,RU,1579466463,85,-3.12,71.69,128.87,8.57
2,2,dombas,100,NO,1579466463,84,14.85,62.07,9.12,5.32
3,3,nanortalik,100,GL,1579466221,83,24.04,60.14,-45.24,27.58
4,4,blythe,1,US,1579466323,15,68.00,33.61,-114.60,10.29


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# Configure gmaps
gmaps.configure(api_key=Google_Maps_Places)

# Store 'Lat' and 'Lng' from DF into lists
location = vacation[["Lat", "Lng"]]
humidity = vacation["Humidity"].astype(float)


In [7]:
# Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

# Add heat layer over map 
heat_layer = gmaps.heatmap_layer(location, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat_layer)
#Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
# configuration of perfect conditions (my perfect conditions)
ideal_temp=vacation.loc[(vacation['Max Temp']>=70)&(vacation['Max Temp']<=80)]
ideal_vacay=temp_file.loc[(temp_file['Wind Speed']<=25)]
ideal_vacay.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Max Temp,Lat,Lng,Wind Speed
14,14,rikitea,100,PF,1579466464,78,78.12,-23.12,-134.97,13.13
34,34,nouadhibou,89,MR,1579466237,31,73.40,20.93,-17.03,11.41
36,36,kapaa,75,US,1579466278,57,75.20,22.08,-159.32,14.99
58,58,vao,9,NC,1579466470,69,75.49,-22.67,167.48,16.17
61,61,hilo,75,US,1579466470,69,75.99,19.73,-155.09,5.82


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
#setting up hotel dataframe
hotel_df = ideal_vacay.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

#params i want
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": Google_Maps_Places
}
#lat and lng to identify hotels
for index, row in hotel_df.iterrows():
    
    #latitudes and longitudes from hotrel df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration wwith params
    params["location"] = f"{lat},{lng}"
    
    #using search terms and lats and lngs
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #make request print
    hotel_name = requests.get(url, params=params)
    
    #using json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    #try/except clause so my code doesnt stop
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -23.1276524,
                    "lng": -134.9656596
                },
                "viewport": {
                    "northeast": {
                        "lat": -23.12637127010728,
                        "lng": -134.9643498701072
                    },
                    "southwest": {
                        "lat": -23.12907092989272,
                        "lng": -134.9670495298927
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "45da19509c7981427f6f1e9583a48b16fcc0c451",
            "name": "Pension Bianca & Benoit",
            "photos": [
                {
                    "height": 2048,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/11515043980962209

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 22.0567505,
                    "lng": -159.3224762
                },
                "viewport": {
                    "northeast": {
                        "lat": 22.05819567989272,
                        "lng": -159.3212718701072
                    },
                    "southwest": {
                        "lat": 22.05549602010728,
                        "lng": -159.3239715298927
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "f8c25eee22f1e5a38196ebdaf03f22af78873c6f",
            "name": "Sheraton Kauai Coconut Beach Resort",
            "photos": [
                {
                    "height": 1333,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/11226162

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -22.6644883,
                    "lng": 167.4485704
                },
                "viewport": {
                    "northeast": {
                        "lat": -22.66328637010728,
                        "lng": 167.4500703798927
                    },
                    "southwest": {
                        "lat": -22.66598602989272,
                        "lng": 167.4473707201073
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "e4fa49be61e064cfb83efd2317b13c2a29101123",
            "name": "OURE TERA beach resort",
            "photos": [
                {
                    "height": 1366,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/116924602195758624807

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -30.1920293,
                    "lng": -50.2096733
                },
                "viewport": {
                    "northeast": {
                        "lat": -30.19067157010728,
                        "lng": -50.20835117010726
                    },
                    "southwest": {
                        "lat": -30.19337122989273,
                        "lng": -50.21105082989271
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "815f32d75ec2664a2a3122188e330b5def74ac76",
            "name": "Pousada Rosa",
            "photos": [
                {
                    "height": 1080,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/111884356728783647286\">Cryst

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAALQW9tVvdsrjMo57crNBcSEeZr29rajsYW8dwv6EnqiCtm_zTuAJQSbhXkJSAojwvCC8brAAtP8hzeOk_RMpfvoWQDm1S0tuGGOwgXk8P1pFmkTpPLq0ESwRpLLlDR8xXazT9hHlmXcGLcth5bOz6eSNOVzfCo92JF7Ib1rsHKYLAOYws6AzPjELx60YINcK7_hD1WnW7uTRyjpX_u5ZfFlgRUfsePII2PfLoKnM7vV1pfGyxVFfT7mQrDjwtzDtzBwuCwHIWskyP_PFs-8msVAmyfhvgwJZEMgWC9ij-7G_Pg7kiKPCVeQdl-zEEsuDvSpdLROBvOOGPvZWtSSnOpxbA8wyLJog_DD4FmLwxcUNoZQoyLyS1ohTzi3yU1esJHVaWP0gAHnmwClttKCxDuESEET_xfgkpZZ3PCafafKHjBEaFLAzAlT1UvXVKyl7tJbukfnlxTOS",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -22.9654529,
                    "lng": -42.0293542
                },
                "viewport": {
                    "northeast": {
                        "lat": -22.96403572010727,
                        "lng": -42.02805672010727
                    },
                    "southwest": {
                        "lat": -22.96673537989272,
                  

...

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAM1mAxnUNMKdJ7oDw_C_P2SueGGpmvj_ulnADjWrR-h4Q7xOhjvEekn6qK6NE6OQ0zrgpaZ8orrnpwal7oqH_HRC24X93kM7hZ_1B5c_pMzuiK2pPDMs_l6XmHhMokOajWTLBKV1i8pBs_z4Q6xhhTiG50z1F9PqTCEr_ovLBRGQg9_mfRS8PBKVmEjaM0q3i2hZOtXTPixBdWKEgq6PxwPCG2Xxjy7_2LZbuDgwAA2Fu34HptE_EVhng8Y4XV5l0eFvunWM9J0vjd6mi7iZtqtQWDyW3u-giGksjb3HIDc6j18d01gP6cfJUlap1ZTaoErId9qsdmE6eSy6C6jjK4S-HVdROU-c1k0CrKNCDFM4koF7_96gklmHTQQE88bqF6T9chjbCVlD8sTLkou3L4gSEIduLnWPuTMLy9SLKj9_L0oaFL9TQc1vLFgIHZwp_sgKniIr_wz3",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 5.939708,
                    "lng": 80.500931
                },
                "viewport": {
                    "northeast": {
                        "lat": 5.941262129892722,
                        "lng": 80.50222297989272
                    },
                    "southwest": {
                        "lat": 5.938562470107279,
                        "l

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -19.544994,
                    "lng": 169.2716525
                },
                "viewport": {
                    "northeast": {
                        "lat": -19.54297052010728,
                        "lng": 169.276127
                    },
                    "southwest": {
                        "lat": -19.54567017989272,
                        "lng": 169.270161
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "40a1134da2892e78e4ea3c62c53f7b5a30ed6250",
            "name": "Apera Beach Front Bungalows",
            "photos": [
                {
                    "height": 3024,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/105443621168952504770\">Florenc

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 26.5462522,
                    "lng": -77.046387
                },
                "viewport": {
                    "northeast": {
                        "lat": 26.54770512989272,
                        "lng": -77.04509337010728
                    },
                    "southwest": {
                        "lat": 26.54500547010728,
                        "lng": -77.04779302989273
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "a89a39cacb7d461e30ffbc134d381e226345f3a0",
            "name": "Abaco Beach Resort",
            "photos": [
                {
                    "height": 3366,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/115815780323722263147\">A G

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 27.4128934,
                    "lng": -82.57382109999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 27.41447927989272,
                        "lng": -82.57215847010727
                    },
                    "southwest": {
                        "lat": 27.41177962010728,
                        "lng": -82.5748581298927
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "50b6173c3c5646b35d8380654ad2d0ffc856f9f1",
            "name": "Ramada by Wyndham Sarasota",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 2736,
                    "html_attributions": [
     

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Max Temp,Lat,Lng,Wind Speed,Hotel Name
0,14,rikitea,100,PF,1579466464,78,78.12,-23.12,-134.97,13.13,Pension Bianca & Benoit
1,34,nouadhibou,89,MR,1579466237,31,73.40,20.93,-17.03,11.41,HOtel Tasiast
2,36,kapaa,75,US,1579466278,57,75.20,22.08,-159.32,14.99,Sheraton Kauai Coconut Beach Resort
3,58,vao,9,NC,1579466470,69,75.49,-22.67,167.48,16.17,OURE TERA beach resort
4,61,hilo,75,US,1579466470,69,75.99,19.73,-155.09,5.82,Grand Naniloa Hotel Hilo - a DoubleTree by Hilton
5,63,cidreira,26,BR,1579466222,80,75.29,-30.18,-50.21,24.45,Pousada Rosa
6,65,arraial do cabo,40,BR,1579466471,69,78.80,-22.97,-42.03,9.17,Hotel da Canoa
7,75,mapua,74,NZ,1579466472,48,77.00,-41.25,173.10,4.00,Pine Hill Lodge & Pine Hill Serenity Vineyard
8,90,axim,0,GH,1579466474,83,79.65,4.87,-2.24,4.70,Ankobra Beach Resort
9,94,edd,0,ER,1579466474,72,75.04,13.93,41.69,3.33,


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [22]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(location,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)

# Display Map
fig


Figure(layout=FigureLayout(height='420px'))